# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 1:06PM,258381,20,9217682,"Exact-Rx, Inc.",Released
1,Feb 28 2023 1:06PM,258381,20,9217681,"Exact-Rx, Inc.",Released
2,Feb 28 2023 12:45PM,258380,20,9217153,"Exact-Rx, Inc.",Released
3,Feb 28 2023 12:45PM,258380,20,9217154,"Exact-Rx, Inc.",Released
4,Feb 28 2023 12:45PM,258380,20,9217155,"Exact-Rx, Inc.",Released
5,Feb 28 2023 12:44PM,258379,21,WireDecking3,"NBTY Global, Inc.",Executing
6,Feb 28 2023 12:28PM,258377,10,SOTAH0000920,"Nextsource Biotechnology, LLC",Released
7,Feb 28 2023 12:28PM,258377,10,SOTAH0000922,"Nextsource Biotechnology, LLC",Released
8,Feb 28 2023 12:28PM,258377,10,SOTAH0000921,"Nextsource Biotechnology, LLC",Released
9,Feb 28 2023 12:26PM,258376,19,ELI030123,Eli Lilly and Company,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,258376,Released,1
42,258377,Released,3
43,258379,Executing,1
44,258380,Released,3
45,258381,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258376,NaN,NaN,1.0
258377,NaN,NaN,3.0
258379,NaN,1.0,NaN
258380,NaN,NaN,3.0
258381,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,1.0,6.0,0.0
258251,1.0,17.0,2.0
258260,3.0,14.0,4.0
258267,10.0,19.0,3.0
258281,7.0,11.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258225,1,6,0
258251,1,17,2
258260,3,14,4
258267,10,19,3
258281,7,11,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,1,6,0
1,258251,1,17,2
2,258260,3,14,4
3,258267,10,19,3
4,258281,7,11,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258225,1,6,
1,258251,1,17,2
2,258260,3,14,4
3,258267,10,19,3
4,258281,7,11,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc."
2,Feb 28 2023 12:45PM,258380,20,"Exact-Rx, Inc."
5,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc."
6,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC"
9,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company
10,Feb 28 2023 12:21PM,258373,10,Emerginnova
14,Feb 28 2023 11:55AM,258371,10,Bio-PRF
15,Feb 28 2023 11:52AM,258370,19,ACG North America LLC
16,Feb 28 2023 11:26AM,258368,10,Eywa Pharma Inc.
29,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",,,2
1,Feb 28 2023 12:45PM,258380,20,"Exact-Rx, Inc.",,,3
2,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",,1,
3,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",,,3
4,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,,,1
5,Feb 28 2023 12:21PM,258373,10,Emerginnova,,,4
6,Feb 28 2023 11:55AM,258371,10,Bio-PRF,,,1
7,Feb 28 2023 11:52AM,258370,19,ACG North America LLC,,,1
8,Feb 28 2023 11:26AM,258368,10,Eywa Pharma Inc.,,,13
9,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2,,
1,Feb 28 2023 12:45PM,258380,20,"Exact-Rx, Inc.",3,,
2,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",,1,
3,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3,,
4,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1,,
5,Feb 28 2023 12:21PM,258373,10,Emerginnova,4,,
6,Feb 28 2023 11:55AM,258371,10,Bio-PRF,1,,
7,Feb 28 2023 11:52AM,258370,19,ACG North America LLC,1,,
8,Feb 28 2023 11:26AM,258368,10,Eywa Pharma Inc.,13,,
9,Feb 28 2023 11:22AM,258367,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2,,
1,Feb 28 2023 12:45PM,258380,20,"Exact-Rx, Inc.",3,,
2,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",,1,
3,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3,,
4,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2.0,NaN,NaN
1,Feb 28 2023 12:45PM,258380,20,"Exact-Rx, Inc.",3.0,NaN,NaN
2,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",NaN,1.0,NaN
3,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
4,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 1:06PM,258381,20,"Exact-Rx, Inc.",2.0,0.0,0.0
1,Feb 28 2023 12:45PM,258380,20,"Exact-Rx, Inc.",3.0,0.0,0.0
2,Feb 28 2023 12:44PM,258379,21,"NBTY Global, Inc.",0.0,1.0,0.0
3,Feb 28 2023 12:28PM,258377,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
4,Feb 28 2023 12:26PM,258376,19,Eli Lilly and Company,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4908631,259.0,1.0,0.0
12,258342,1.0,0.0,0.0
15,1033003,9.0,56.0,15.0
19,1291684,3.0,17.0,10.0
20,775121,13.0,0.0,0.0
21,258379,0.0,1.0,0.0
22,775098,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4908631,259.0,1.0,0.0
1,12,258342,1.0,0.0,0.0
2,15,1033003,9.0,56.0,15.0
3,19,1291684,3.0,17.0,10.0
4,20,775121,13.0,0.0,0.0
5,21,258379,0.0,1.0,0.0
6,22,775098,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,259.0,1.0,0.0
1,12,1.0,0.0,0.0
2,15,9.0,56.0,15.0
3,19,3.0,17.0,10.0
4,20,13.0,0.0,0.0
5,21,0.0,1.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,259.0
1,12,Released,1.0
2,15,Released,9.0
3,19,Released,3.0
4,20,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,15.0,10.0,0.0,0.0,0.0
Executing,1.0,0.0,56.0,17.0,0.0,1.0,0.0
Released,259.0,1.0,9.0,3.0,13.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,15.0,10.0,0.0,0.0,0.0
1,Executing,1.0,0.0,56.0,17.0,0.0,1.0,0.0
2,Released,259.0,1.0,9.0,3.0,13.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,15.0,10.0,0.0,0.0,0.0
1,Executing,1.0,0.0,56.0,17.0,0.0,1.0,0.0
2,Released,259.0,1.0,9.0,3.0,13.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()